In [2]:
import torch 
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
words = open('names.txt','r').read().splitlines()
len(words)

32033

In [4]:
chars =sorted(list(set(''.join(words))))
sti={s:i+1 for i,s in enumerate(chars)} 
sti['.']=0
its={s:i for i,s in sti.items()}
vocab_size=len(its)

In [5]:
for i in range(len(words)):
    words[i] = words[i] + "."

In [6]:
cont_size=3
X,Y=[],[]
for w in words:
    context=[0]*cont_size
    #print(w)
    for ch in w:
        ix=sti[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(its[i] for i in context),"------->",its[ix])
        context  = context[1:]+[ix]
X=torch.tensor(X)
Y=torch.tensor(Y)

In [7]:
X,Y

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         ...,
         [26, 26, 25],
         [26, 25, 26],
         [25, 26, 24]]),
 tensor([ 5, 13, 13,  ..., 26, 24,  0]))

In [8]:
cont_size=3

def build_ds(words):
    X,Y=[],[]
    for w in words:
        context=[0]*cont_size
        #print(w)
        for ch in w:
            ix=sti[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(its[i] for i in context),"------->",its[ix])
            context  = context[1:]+[ix]
    X=torch.tensor(X)
    Y=torch.tensor(Y)
    return X,Y
import random
random.seed(42)
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))
Xtr,Ytr = build_ds(words[:n1])
Xva,Yva = build_ds(words[n1:n2])
Xte,Yte = build_ds(words[n2:])

In [9]:
C=torch.rand([27,2])
C

tensor([[0.8598, 0.5601],
        [0.3771, 0.0793],
        [0.9796, 0.7679],
        [0.9292, 0.3725],
        [0.8713, 0.4423],
        [0.5786, 0.4977],
        [0.6446, 0.3617],
        [0.6959, 0.8241],
        [0.0623, 0.7495],
        [0.1709, 0.7111],
        [0.5432, 0.1298],
        [0.9452, 0.3008],
        [0.2042, 0.0314],
        [0.4019, 0.8868],
        [0.5151, 0.5126],
        [0.4524, 0.1777],
        [0.7004, 0.5837],
        [0.5662, 0.8691],
        [0.7786, 0.8137],
        [0.5221, 0.8534],
        [0.1046, 0.1352],
        [0.5555, 0.2828],
        [0.6722, 0.8154],
        [0.2012, 0.1752],
        [0.7905, 0.1230],
        [0.2910, 0.7636],
        [0.1158, 0.9782]])

In [10]:
Xtr.shape,Ytr.shape

(torch.Size([182625, 3]), torch.Size([182625]))

In [11]:
#MLP
n_emb = 10 # the dimensionality of the lookup/embedding matrix
n_hidden = 200
g = torch.Generator().manual_seed(2147483647)
C=torch.rand((vocab_size,n_emb),                generator = g)
w1=torch.randn((n_emb*cont_size,n_hidden),      generator = g) * ((5/3)/(30**0.5))#six because we give 3 inputs encoded in 2D space!!!read about it once more!!
b1=torch.randn(n_hidden,                        generator = g) * 0.01
w2=torch.randn((n_hidden,vocab_size),            generator = g) * 0.01
b2=torch.randn(vocab_size,                      generator = g) * 0 
bnmean_run =torch.zeros((1,n_hidden))
bnstd_run = torch.ones((1,n_hidden)) 
bngain = torch.ones((1,n_hidden))
bnbias=torch.zeros((1,n_hidden))
parameters = [C,w1,b1,w2,b2]
print("The number of parameters in the current Model is: ",sum(p.nelement() for p in parameters))

The number of parameters in the current Model is:  11897


In [12]:
#torch.cat(torch.unbind(emb,1),1)
(5/3)/n_emb*cont_size**0.5 #read about gain function and kaming kit

0.2886751345948129

In [13]:
for p in parameters:
    p.requires_grad = True

In [14]:
#lre=torch.linspace(-3,0,1000)
#lrs=10**lre
#lrs

In [20]:
class Linear:
    def __init__(self,fanin,fanout,bias=True):
        self.weight = torch.randn((fanin,fanout),generator=g)/fanin**0.5
        self.bias = torch.zeros(fanout) if bias else None
    def __call__(self,x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out+=self.bias
        return self.out
    def parameters(self):
        return [self.weight]+([] if self.bias is None else [self.bias])
class BatchNorm1d:
    def __init__(self,dim,eps=1e-5,momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        #parameters(trained with backprop)
        self.gamma = torch.ones(dim)#gain
        self.beta = torch.zeros(dim)#bias
        #buffers
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)


    def __call__(self,x):
        if self.training:
            xmean = x.mean(0,keepdim = True)
            xvar = x.var(0,keepdim = True,unbiased=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x-xmean) / torch.sqrt(xvar+self.eps)
        self.out = self.gamma*xhat + self.beta

        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    def parameters(self):
        return [self.gamma , self.beta]
class Tanh:
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []
n_embd = 10
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)
C=torch.rand((vocab_size,n_emb),                generator = g)
layers=[
    Linear(n_emb * cont_size, n_hidden),BatchNorm1d(n_hidden), Tanh(),     
    Linear(n_hidden,        n_hidden),  BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden,        n_hidden),  BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden,        n_hidden),  BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden,        n_hidden),  BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden,      vocab_size),  BatchNorm1d(vocab_size),
]
with torch.no_grad():
    #layers[-1].weight*=0.1 #making last leyer less confident
    layers[-1].gamma*=0.1
    for layer in layers[:-1]:
        if isinstance(layer,Linear):
            layer.weight*= 5/3
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print("Total number of tunable parameters are",sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

Total number of tunable parameters are 47551


In [ ]:
max_steps = 200001
batch_size=32
lossi=[]

for i in range(max_steps):
    #constructing a mini batches so computation is easier
    ix = torch.randint(0,Xtr.shape[0],(batch_size,),generator = g)
    Xe,Ye  = Xtr[ix],Ytr[ix]
    #forward pass
    emb = C[Xe]
    x = emb.view(emb.shape[0],-1)######################!!!!!!!!!!!!!!111
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x,Ye)
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()
    lr=0.1 if i < 100000 else 0.01 # enabling step wise gradient decay
    for p in parameters:
        #p.data += -.1 * p.grad
        p.data += -lr * p.grad   #gradient decay so just a small push for the model at the end

    #tracking
    if i  % 10000 == 0:
        print(f"{i:7d}/{max_steps-1:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())

      0/ 200000: 3.3018
  10000/ 200000: 2.2993
  20000/ 200000: 2.3659
  30000/ 200000: 2.1266
  40000/ 200000: 2.0976
  50000/ 200000: 2.2632
  60000/ 200000: 1.9850


In [ ]:
import numpy as np
plt.figure(figsize=(20,4))#width and height
legends = []
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer,Tanh):
        t=layer.out
        print("layer %d (%10s): mean%+.2f, std %.2f, saturated: %.2f%%" % (i,layer.__class__.__name__,t.mean(),t.std(),(t.abs()>0.97).float().mean()*100))
        hy,hx = torch.histogram(t,density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'layer {i} ({layer.__class__.__name__})')
plt.legend(legends);
plt.title("Activation Distribution")

In [ ]:
import numpy as np
plt.figure(figsize=(20,4))#width and height
legends = []
for i,layer in enumerate(layers[:-1]):
    if isinstance(layer,Tanh):
        t=layer.out.grad
        print("layer %d (%10s): mean%+f, std %e, saturated: %.2f%%" % (i,layer.__class__.__name__,t.mean(),t.std(),(t.abs()>0.97).float().mean()*100))
        hy,hx = torch.histogram(t,density=True)
        plt.plot(hx[:-1].detach(), hy.detach())
        legends.append(f'layer {i} ({layer.__class__.__name__})')
plt.legend(legends);
plt.title("Activation Distribution")

max_steps = 200001
batch_size=32
lossi=[]

for i in range(max_steps):
    #constructing a mini batches so computation is easier
    ix = torch.randint(0,Xtr.shape[0],(batch_size,))
    Xe,Ye  = Xtr[ix],Ytr[ix]
    #forward pass
    emb = C[Xe]
    embcat = emb.view(emb.shape[0],-1)######################!!!!!!!!!!!!!!111
    hpreact = embcat @ w1 +#b1  #we dont need bias here as we are subtracting bnmean again so its just useless
    bnmeani=hpreact.mean(0,keepdim = True)
    bnstdi=hpreact.std(0,keepdim = True)
    hpreact = bngain*(hpreact-bnmeani)/bnstdi+bnbias
    
    
    with torch.no_grad():
            bnmean_run = 0.99*bnmean_run+0.001*bnmeani
            bnstd_run = 0.99*bnstd_run+0.001*bnstdi
            #print(f"Inside the 'with' block, loss.requires_grad is: {loss.requires_grad}")
    h=torch.tanh(hpreact) # why -1 e3 at 26:00
    #print(f"Inside the 'with' block, loss.requires_grad is: {loss.requires_grad}")

    logits=h @ w2 + b2
    loss = F.cross_entropy(logits,Ye)
    #print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    lr=0.1 if i < 100000 else 0.01 # enabling step wise gradient decay
    for p in parameters:
        #p.data += -.1 * p.grad
        p.data += -lr * p.grad   #gradient decay so just a small push for the model at the end

    #tracking
    if i  % 10000 == 0:
        print(f"{i:7d}/{max_steps-1:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())


In [ ]:
hpreact.std(0,keepdim = True).shape

In [ ]:
hpreact.mean(0,keepdim = True).shape

In [ ]:
plt.hist(h.view(-1).tolist(), 50);

In [ ]:
plt.hist(hpreact.view(-1).tolist(), 50);

In [ ]:
plt.plot(lossi)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(h.abs()>0.99,cmap='gray',interpolation='nearest')


# loss =-prob[torch.arange(67),Y].log().mean()
# other way to calculate loss is through entropy function
# reasons to call cross entropy : ...
# loss = F.cross_entropy(logits,Y)
# loss#!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
with torch.no_grad():
    # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ w1 + b1
    # measure the mean/std over the entire training set
    bnmean = hpreact.mean(0, keepdim=True)
    bnstd = hpreact.std(0, keepdim=True)

In [ ]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        "train":(Xtr,Ytr),
        "val":(Xva,Yva),
        "test":(Xte,Yte),
    }[split]
    emb = C[X]
    x = emb.view(emb.shape[0],-1)######################!!!!!!!!!!!!!!111
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x,Y)
    print(split,loss.item())
split_loss("train")
split_loss("val")

In [ ]:
g = torch.Generator().manual_seed(2147483647+10)
for i in range(10):
    out = []
    context = [0] * cont_size
    while True:
        emb = C[torch.tensor([context])]
        embcat = emb.view(1,-1)######################!!!!!!!!!!!!!!111
        hpreact = embcat @ w1 +b1
        h=torch.tanh(hpreact) # why -1 e3 at 26:00
        logits=h @ w2 + b2
        probs = F.softmax(logits,dim=1)
        ix = torch.multinomial(probs,num_samples = 1,generator =g).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    print(''.join(its[i] for i in out))

In [ ]:
nn.Linear(n_emb * cont_size, n_hidden), nn.Tanh(),     
    nn.Linear(n_hidden,        n_hidden),   nn.Tanh(),
    nn.Linear(n_hidden,        n_hidden),   nn.Tanh(),
    nn.Linear(n_hidden,        n_hidden),   nn.Tanh(),
    nn.Linear(n_hidden,        n_hidden),   nn.Tanh(),
    nn.Linear(n_hidden,      vocab_size),

In [ ]:
### import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 1. Load Data
try:
    words = open('names.txt', 'r').read().splitlines()
except FileNotFoundError:
    # Fallback if file not found (though user uploaded it)
    words = ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

# 2. Build Dataset
block_size = 3
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

X, Y = build_dataset(words)

# 3. Initialize Model (Embedding size = 2 for visualization)
n_embd = 2
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

# 4. Train (Quick Run for visualization)
# We need enough steps to separate the characters somewhat nicely
steps = 5000 
for i in range(steps):
    ix = torch.randint(0, X.shape[0], (32,))
    
    # Forward pass
    emb = C[X[ix]] 
    embcat = emb.view(emb.shape[0], -1) 
    hpreact = embcat @ W1 + b1
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    
    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update
    lr = 0.1 if i < steps/2 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

# 5. Plot Embeddings
plt.figure(figsize=(10, 10))
plt.scatter(C[:,0].data, C[:,1].data, s=200, alpha=0.6, edgecolors='white')
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='black', fontsize=12, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.5)
plt.title('Learned Character Embeddings (2D Projection)', fontsize=16)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')

# Save
plt.savefig('embeddings_plot.png', dpi=150, bbox_inches='tight')